# Food Inflation Dynamics in the United States (1974–2024)


*Food price inflation plays a central role in household welfare, food affordability, and nutritional outcomes. Using historical Consumer Price Index (CPI) and Producer Price Index (PPI) data from 1974 to 2024, we examine how food inflation has evolved over time, how it differs across major economic periods, and how inflation dynamics vary across food categories. We combine descriptive time-series analysis with rolling volatility measures, persistence metrics, and trend-based models to characterize inflation regimes and category-level heterogeneity. Our results show that food inflation is strongly regime-dependent, with elevated levels and volatility during the late 1970s and the post-COVID period, and relatively stable behavior during the intervening decades. At the category level, inflation dynamics differ substantially in both volatility and persistence, reflecting heterogeneous supply-side exposure. While unhealthy or processed foods exhibit slightly higher average inflation than healthier categories, the difference is small and not statistically significant over the long run. Together, these findings highlight the importance of structural and supply-driven forces in shaping food price inflation rather than uniform trends across food types.*

## Introduction and Motivation
Food prices affect nearly all households and disproportionately impact lower-income consumers, making food inflation a key economic and policy concern. Periods of elevated food inflation—such as the late 1970s, the Great Recession, and the post-COVID inflation surge—have drawn attention to the drivers and persistence of food price changes. At the same time, food price dynamics vary across categories due to differences in production processes, perishability, regulation, and exposure to global shocks.
This project investigates long-run and short-run food inflation dynamics in the United States using historical CPI and PPI data. Rather than focusing on causal mechanisms, we adopt an exploratory and descriptive approach to answer the following questions:
1. Which food categories have experienced the fastest price increases over the past 20–50 years?
2. Do healthier food categories experience higher inflation than unhealthy or processed categories?
3. How does food inflation differ across major economic periods such as the 1970s inflation era, the Great Recession, COVID-19, and the post-COVID inflation surge and which food categories exhibit the greatest volatility and persistence in inflation?
By combining aggregate and category-level analyses, we aim to provide a comprehensive picture of food inflation dynamics across time and economic regimes.

## Data and Assumptions
We use annual percent-change data from the U.S. Consumer Price Index (CPI) and Producer Price Index (PPI), published by the Bureau of Labor Statistics and compiled by the USDA Economic Research Service. The CPI measures changes in prices paid by consumers, while the PPI captures price changes at earlier stages of the supply chain, such as farm-level and wholesale prices. Both datasets span 1974–2024 and report annual inflation rates by food category.
Because the data are observational and aggregated at the annual level, our analysis does not attempt causal inference. Annual aggregation smooths seasonal fluctuations but may obscure short-term shocks. Additionally, food category definitions are broad and may mask within-category heterogeneity. When classifying foods as “healthy” or “unhealthy,” we rely on a transparent but necessarily subjective grouping, which we treat as an analytical device rather than a normative judgment.

**Aggregate Food Inflation Over Time**

We begin by examining aggregate food inflation measured by the CPI “All Food” category. The time series reveals pronounced spikes during the late 1970s and early 1980s, followed by a long period of relatively low and stable inflation, and a renewed surge beginning after 2020.
To assess whether the long-run trend in food inflation has changed over time, we estimate a broken-stick (change-of-slope) regression. This model allows the trend in inflation to evolve gradually rather than imposing a single linear trend. The estimated breakpoint occurs around 1985, marking a transition from a declining inflation trend to a flatter long-run pattern. The broken-stick model fits the data substantially better than a simple linear trend, suggesting that food inflation regimes evolve smoothly rather than through sharp structural breaks.
Formal change-point detection methods corroborate this interpretation: at conservative penalty levels, no statistically sharp breaks are detected in aggregate food inflation. Together, these results indicate that food inflation is shaped by slow-moving structural forces rather than abrupt, discrete regime shifts.

**Food Inflation Across Major Economic Periods**

To better understand regime-specific behavior, we group years into economically meaningful periods: the 1970s inflation era, the Great Recession (2008–2009), the COVID-19 period (2020–2021), the post-COVID inflation surge (2022–2023), and a baseline period covering the remaining years.
Average inflation and volatility differ markedly across these regimes. The 1970s and post-COVID periods exhibit both high average inflation and elevated volatility, while the baseline period is characterized by low and stable inflation. The COVID-19 period shows moderately elevated inflation but comparatively low volatility, reflecting a short-lived but concentrated shock.
Distributional comparisons further reveal that high-inflation regimes are associated with wider dispersion in annual inflation rates, underscoring the increased uncertainty households face during such periods. These findings emphasize that food inflation cannot be characterized by a single long-run average; instead, its behavior depends strongly on broader macroeconomic conditions.

**Volatility, Persistence, and CPI–PPI Linkages**

Beyond average inflation levels, volatility and persistence provide insight into how inflation shocks evolve over time. Rolling five-year volatility measures show that food inflation volatility clusters around major economic disruptions, particularly the late 1970s and the post-COVID period.
Comparing CPI and PPI volatility reveals strong co-movement between consumer- and producer-level price instability. The correlation between rolling CPI and PPI volatility exceeds 0.8, suggesting that food price volatility is largely driven by common supply-side shocks rather than category-specific consumer dynamics.
Persistence analysis further distinguishes consumer and producer inflation behavior. CPI food inflation exhibits substantially higher year-to-year persistence than PPI inflation, indicating that consumer price shocks tend to linger longer than producer-level shocks. This pattern suggests frictions in price adjustment or pass-through mechanisms that prolong consumer-level inflation.

**Category-Level Inflation Dynamics**

Aggregate measures can conceal substantial heterogeneity across food categories. Analyzing major food groups separately reveals pronounced differences in inflation trends, volatility, and persistence.
Broken-stick regressions applied at the category level identify different trend breakpoints across food types, indicating that inflation regimes evolve asynchronously. For example, some categories exhibit trend shifts in the mid-1970s, while others change much later, reflecting category-specific supply chains and exposure to shocks.
Volatility analysis highlights fats and oils and meats as particularly unstable categories, while cereals and bakery products exhibit lower average volatility. Correlation analyses of rolling volatility show that some categories experience synchronized volatility spikes, consistent with system-wide shocks, while others display more idiosyncratic behavior.
Together, these results demonstrate that food inflation is not monolithic: different food categories face distinct inflation risks driven by heterogeneous structural and supply-side factors.

**Healthy vs. Unhealthy Food Inflation**

To assess whether healthier foods experience systematically higher inflation, we classify food categories into “healthy” (e.g., fruits, vegetables, dairy, meats) and “unhealthy or processed” (e.g., sugars, fats and oils, nonalcoholic beverages). Over the 1974–2022 period, unhealthy categories exhibit a higher average annual inflation rate (approximately 4.30%) than healthy categories (approximately 3.70%), a difference of about 0.61 percentage points.
Unhealthy categories also display greater volatility, with a larger standard deviation of annual inflation rates. However, formal statistical tests do not detect a statistically significant difference in mean inflation between the two groups, and the estimated effect size is negligible. These findings suggest that while unhealthy foods have tended to inflate slightly faster, the long-run difference is modest relative to the overall variability in food inflation.

**Fastest-Inflating Food Categories**

Ranking categories by average annual inflation highlights additional heterogeneity. Over the full 1974–2024 period, CPI data show that sugar and sweets, fresh fruits, cereals and bakery products, and food away from home are among the fastest-inflating categories. In contrast, recent decades show a shift toward animal-based products such as beef and eggs.
At the producer level, PPI data reveal even more pronounced inflation for farm-level eggs and certain agricultural inputs, particularly in the post-2000 period. Comparing CPI and PPI rankings suggests that upstream price pressures play a central role in shaping consumer food inflation, though the transmission varies by category.

## Discussion
Taken together, our findings highlight three central themes. First, food inflation is strongly regime-dependent, with elevated levels and volatility during periods of macroeconomic stress. Second, food inflation dynamics are largely driven by supply-side forces, as evidenced by strong CPI–PPI co-movement and higher persistence at the consumer level. Third, substantial heterogeneity exists across food categories, implying that aggregate measures obscure important distributional and risk considerations.
While unhealthy foods have experienced slightly higher average inflation, the difference is small relative to the overall variability of food prices. This suggests that concerns about food affordability during inflationary periods should focus not only on average price changes, but also on volatility, persistence, and category-specific exposure.

## Conclusion
This project provides a comprehensive descriptive analysis of U.S. food inflation from 1974 to 2024. Food inflation exhibits clear regime-dependent behavior, substantial category-level heterogeneity, and strong links between producer and consumer price dynamics. Rather than evolving through sharp structural breaks, food inflation trends change gradually in response to long-run economic forces. These findings underscore the importance of considering both aggregate and disaggregated perspectives when evaluating food price inflation and its implications for consumers.

## Author Contributions

* Alistair Clark: Analyzed inflation differences between healthy and unhealthy food categories.
* Emily Hsu: Conducted the analysis of the fastest-inflating food categories using CPI and PPI data.
* Fiona Li: Analysis of food inflation across major economic periods.
* Thomas Lee: Data preprocessing, ranking analyses, visualization, reproducibility tools.
  
*All authors reviewed and approved the final manuscript.*